In [ ]:
pip install ibm_db

In [ ]:
import ibm_db as db

In [ ]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "1bbf73c5-d84a-4bb0-85b9-ab1a4348f4a4.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "myt42210"        # e.g. "abc12345"
dsn_pwd = "BicTxYRz4Yv3pLVX"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"            # e.g. "BLUDB"
dsn_port = "32286"                # e.g. "32733"
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", db.conn_errormsg() )


In [ ]:
#Retrieve Metadata for the Database Server
server = db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

In [ ]:
#Retrieve Metadata for the Database Client / Driver
client = db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME)
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

SQL Connection Finished ABOVE


In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
import pandas as pd
data=pd.read_csv("psql.csv")
df=pd.DataFrame(data)
df.head()

In [ ]:
import ibm_db as db
import pandas as pd

In [ ]:
df.shape

In [ ]:
d="select * from CHI"
X=db.exec_immediate(conn,d)

In [ ]:
print(X)

In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

In [ ]:
select="select count(*) from CHI"
sel=db.exec_immediate(conn,select)
db.fetch_both(sel)

In [ ]:
#How many community areas in Chicago have a hardship index greater than 50.0?
hr='select count (*) from CHI where HARDSHIP_INDEX > 50.0'
hrr=db.exec_immediate(conn,hr)
db.fetch_both(hrr)

#Correct answer is 38

In [ ]:
#What is the maximum value of hardship index in this dataset?
d='select max(HARDSHIP_INDEX) from CHI'
dd=db.exec_immediate(conn,d)
db.fetch_both(dd)

In [ ]:
#Which community area which has the highest hardship index?
h='select COMMUNITY_AREA_NAME from CHI where HARDSHIP_INDEX = 98.0'
hh=db.exec_immediate(conn,h)
db.fetch_both(hh)


#Other solutions

#or another option:
%sql SELECT community_area_name FROM chicago_socioeconomic_data ORDER BY hardship_index DESC NULLS LAST FETCH FIRST ROW ONLY;

#or you can use a sub-query to determine the max hardship index:
%sql select community_area_name from chicago_socioeconomic_data where hardship_index = ( select max(hardship_index) from chicago_socioeconomic_data )

In [ ]:
#What are the top 10 schools with the highest "Average Student Attendance
%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10

In [ ]:
#Which Chicago community areas have per-capita incomes greater than $60,000?
c='select COMMUNITY_AREA_NAME from CHI where PER_CAPITA_INCOME >  60000'
cc=db.exec_immediate(conn,c)
a=db.fetch_both(cc)

In [ ]:
#Which Schools have Average Student Attendance lower than 70%
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

In [ ]:
#Get the total College Enrollment for each Community Area
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name


In [ ]:
#Get the 5 Community Areas with the least total College Enrollment sorted in ascending order
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

In [ ]:
#List 5 schools with lowest safety score
%sql SELECT name_of_school, safety_score \
FROM schools \
ORDER BY safety_score \
LIMIT 5

In [ ]:
#Get the hardship index for the community area which has College Enrollment of 4368
%%sql
select hardship_index
   from chicago_socioeconomic_data CD, schools CPS
   where CD.ca = CPS.community_area_number
      and college_enrollment = 4368

In [ ]:
#Get the hardship index for the community area which has the school with the highest enrollment
%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

QUerying Real World Datasets

In [ ]:
IBMgift2022



IBM Connection Plan

In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

In [ ]:
%load_ext sql

In [ ]:
# Remember the connection string is of the format:
%sql ibm_db_sa://myt42210:BicTxYRz4Yv3pLVX@1bbf73c5-d84a-4bb0-85b9-ab1a4348f4a4.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:32286/bludb?security=SSL
# Enter the connection string for your Db2 on Cloud database instance below
#%sql ibm_db_sa://

In [ ]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "1bbf73c5-d84a-4bb0-85b9-ab1a4348f4a4.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "myt42210"        # e.g. "abc12345"
dsn_pwd = "BicTxYRz4Yv3pLVX"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"            # e.g. "BLUDB"
dsn_port = "32286"                # e.g. "32733"
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"